In [1]:
## import nbimporter

import pandas as pd
import argparse
import sklearn
import matplotlib.pyplot as plt
import numpy as np
random_seed = 99
import pandas as pd
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
np.random.seed(random_seed)
from keras.optimizers import Adam
from keras.layers import Dense, Flatten, Dropout, Activation, Conv1D, Permute, RepeatVector, multiply, Bidirectional
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import LSTM
from keras.models import Sequential, load_model, Model
from keras.regularizers import l2
from keras.layers import Dense, Dropout, Input, Lambda
from keras import backend as K
import tensorflow as tf
from keras.backend.tensorflow_backend import set_session
config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.3
set_session(tf.Session(config=config))
from __future__ import print_function, division
from dataprep4 import SarcasmDataLoader



class lstm_model:

    def __init__(self, number, hidden_units,dim, vocab_size, max_len, embedding_weights=None):
        self.number = number
        self.hidden_units = hidden_units
        self.dim = dim
        self.vocab_size = vocab_size
        self.max_len = max_len
        self.embedding_weights = embedding_weights

    def loadModel(self):
        input = Input(shape=(self.max_len,))
        embeddings = Embedding(self.vocab_size, self.dim, input_length=self.max_len, weights=[self.embedding_weights], trainable=False)(input)

        
        if self.number == 1:  # LSTM based network
            print('-' * 100)
            print("Model Selected: LSTM based network")
            print('-' * 100)
            lstm_output = LSTM(self.hidden_units)(embeddings)
            lstm_output = Dense(256, activation ='tanh', kernel_initializer='he_normal', kernel_regularizer=l2(0.001))(lstm_output)
            lstm_output = Dropout(0.3)(lstm_output)
            lstm_output = Dense(128, activation ='tanh', kernel_initializer='he_normal', kernel_regularizer=l2(0.001))(lstm_output)
            lstm_output = Dropout(0.3)(lstm_output)
            final_output = Dense(1, activation='sigmoid')(lstm_output)

        elif self.number == 2:  # Bidirectional LSTM 
            print('-' * 100)
            print("Model Selected: Bidirectional LSTM ")
            print('-' * 100)
            lstm_output = Bidirectional(LSTM(self.hidden_units))(embeddings)
            lstm_output = Dense(256, activation ='relu', kernel_initializer='he_normal', kernel_regularizer=l2(0.001))(lstm_output)
            lstm_output = Dropout(0.3)(lstm_output)
            lstm_output = Dense(128, activation ='relu', kernel_initializer='he_normal', kernel_regularizer=l2(0.001))(lstm_output)
            lstm_output = Dropout(0.3)(lstm_output)
            final_output = Dense(1, activation='sigmoid')(lstm_output)

        model = Model(inputs=input, outputs=final_output)
        return model




if __name__ == '__main__':
    # Defining the command line arguments
    epochs = 10
    lr = 1e-03
    batchsize = 32
    number = 2
    
    
    
    # defining the various hyperparameters for the model
    hidden_units = 128  # hidden units for the LSTM
    # the max length of sentence sequence (longer sentences will get truncated
    # to this length)
    max_len = 50

    data_loader_object = SarcasmDataLoader(max_len)
    x_train, y_train, x_test, y_test, vocab_size = data_loader_object.loadData(path='final_clean_twitter.csv')
    print('-' * 100)

    embedding_weights, dim = data_loader_object.loadPreTrainedWordEmbeddings(vocab_size, embedding_path='glove.6B.300d.txt')
    
    sarcasm_model_object = lstm_model(number, hidden_units, dim, vocab_size, max_len,embedding_weights)
    model = sarcasm_model_object.loadModel()

    
    opt = Adam(lr=lr)
    

    #  Compile the model
    model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['acc'])
    
    



Using TensorFlow backend.


----------------------------------------------------------------------------------------------------
----------------------------------------------------------------------------------------------------
Loading the embeddings
GloVe Embeddings loaded
Saved word embeddings loaded
----------------------------------------------------------------------------------------------------
Model Selected: Bidirectional LSTM 
----------------------------------------------------------------------------------------------------


In [2]:
def pred(padded_test):
    if model.predict(padded_test)[0]<0.5:
        print ("-1")
    else:
        print ("1")

In [3]:
model.fit(x_train, y_train, epochs=epochs, batch_size=batchsize,verbose=0, shuffle=True, validation_split=0.1)

In [9]:
tokenizer = Tokenizer()
tokenizer2 = Tokenizer()

In [10]:
import pandas as pd
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

In [11]:
sarcasm_data = pd.read_csv('final_clean_twitter.csv',encoding="utf-8")
seq_matrix = []

In [12]:
number_of_sequences = len(sarcasm_data['body'])
for i in range(number_of_sequences):
    seq_matrix.append(sarcasm_data['body'][i])

In [13]:
tokenizer.fit_on_texts(seq_matrix)
tokenizer2.fit_on_texts(seq_matrix)
#tokenizer3 = tokenizer.fit_on_texts(seq_matrix)
#tokenizer4 = tokenizer.fit_on_texts(seq_matrix)
#tokenizer5 = tokenizer.fit_on_texts(seq_matrix)

In [14]:
encoded_seq_matrix = tokenizer.texts_to_sequences(seq_matrix)

In [15]:
vocab_size = len(tokenizer.word_index) + 1
padded_encoded_seq_matrix = pad_sequences(encoded_seq_matrix, maxlen=50, padding='post')

In [16]:
tweet = ['Not flushing the toilet is okay right']
test_encoded = tokenizer2.texts_to_sequences(tweet)
padded_test = pad_sequences(test_encoded, maxlen=50, padding='post')
pred(padded_test)

1


In [54]:
tweet = ['I cant wait to spend this awesome day at work']
test_encoded = tokenizer2.texts_to_sequences(tweet)
padded_test = pad_sequences(test_encoded, maxlen=50, padding='post')
pred(padded_test)

1


In [55]:
tweet = ['hahahaha']
test_encoded = tokenizer2.texts_to_sequences(tweet)
padded_test = pad_sequences(test_encoded, maxlen=50, padding='post')
pred(padded_test)

1


In [56]:
tweet = ['This is good']
test_encoded = tokenizer2.texts_to_sequences(tweet)
padded_test = pad_sequences(test_encoded, maxlen=50, padding='post')
pred(padded_test)

-1


In [57]:
tweet = ['you are so bad ']
test_encoded = tokenizer2.texts_to_sequences(tweet)
padded_test = pad_sequences(test_encoded, maxlen=50, padding='post')
pred(padded_test)

-1


In [60]:
tweet = ['So happy that I get to work tomorrow ']
test_encoded = tokenizer2.texts_to_sequences(tweet)
padded_test = pad_sequences(test_encoded, maxlen=50, padding='post')
pred(padded_test)

1


In [61]:
tweet = ['Cant wait to work late nights']
test_encoded = tokenizer2.texts_to_sequences(tweet)
padded_test = pad_sequences(test_encoded, maxlen=50, padding='post')
pred(padded_test)

1


In [62]:
tweet = ['New day new beginnings']
test_encoded = tokenizer2.texts_to_sequences(tweet)
padded_test = pad_sequences(test_encoded, maxlen=50, padding='post')
pred(padded_test)

-1


In [63]:
tweet = ['Always be happy']
test_encoded = tokenizer2.texts_to_sequences(tweet)
padded_test = pad_sequences(test_encoded, maxlen=50, padding='post')
pred(padded_test)

-1


In [64]:
tweet = ['Always be happy go lucky']
test_encoded = tokenizer2.texts_to_sequences(tweet)
padded_test = pad_sequences(test_encoded, maxlen=50, padding='post')
pred(padded_test)

-1
